In [1]:
import os
import glob

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks
from keras.optimizers import SGD

import coremltools

Using TensorFlow backend.


In [2]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [3]:
input_shape=(7,7,512)
num_classes=84

In [4]:
model = Sequential()
model.add(Flatten(input_shape=base_model.output_shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('multiclass_50epochs_sgd.h5')

In [5]:
model = Model(inputs= base_model.input, outputs= model(base_model.output))

Save the model

In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
coreml_model  = coremltools.converters.keras.convert(model,
                                                    image_input_names='image',
                                                    class_labels='labels.txt',
                                                    input_names=['image'],  
                                                    red_bias=-123.68,
                                                    green_bias=-116.78,
                                                    blue_bias=-103.94
)

0 : input_1, <keras.engine.topology.InputLayer object at 0x104119e10>
1 : block1_conv1, <keras.layers.convolutional.Conv2D object at 0x104119a50>
2 : block1_conv1__activation__, <keras.layers.core.Activation object at 0x115249e50>
3 : block1_conv2, <keras.layers.convolutional.Conv2D object at 0x104119ed0>
4 : block1_conv2__activation__, <keras.layers.core.Activation object at 0x116b5ad50>
5 : block1_pool, <keras.layers.pooling.MaxPooling2D object at 0x111444c10>
6 : block2_conv1, <keras.layers.convolutional.Conv2D object at 0x11148f1d0>
7 : block2_conv1__activation__, <keras.layers.core.Activation object at 0x116b5ad90>
8 : block2_conv2, <keras.layers.convolutional.Conv2D object at 0x111480190>
9 : block2_conv2__activation__, <keras.layers.core.Activation object at 0x116b5aed0>
10 : block2_pool, <keras.layers.pooling.MaxPooling2D object at 0x11149d710>
11 : block3_conv1, <keras.layers.convolutional.Conv2D object at 0x1114dc510>
12 : block3_conv1__activation__, <keras.layers.core.Activa

In [8]:
coreml_model.save('disdatkerasv4.mlmodel')

## Full model weights

In [11]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

num_classes = 110


# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='sigmoid'))

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

model.load_weights('disdat-110-sgd-decay7-refined.h5')

model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [12]:
coreml_model  = coremltools.converters.keras.convert(model,
                                                    image_input_names='image',
                                                    class_labels='labels.txt',
                                                    input_names=['image'],  
                                                    red_bias=-123.68,
                                                    green_bias=-116.78,
                                                    blue_bias=-103.94
)

0 : input_4, <keras.engine.topology.InputLayer object at 0x7efdeda68510>
1 : block1_conv1, <keras.layers.convolutional.Conv2D object at 0x7efdeda68550>
2 : block1_conv1__activation__, <keras.layers.core.Activation object at 0x7efe2846f590>
3 : block1_conv2, <keras.layers.convolutional.Conv2D object at 0x7efdeed9ead0>
4 : block1_conv2__activation__, <keras.layers.core.Activation object at 0x7efdeda4f950>
5 : block1_pool, <keras.layers.pooling.MaxPooling2D object at 0x7efdeda68650>
6 : block2_conv1, <keras.layers.convolutional.Conv2D object at 0x7efdeda09950>
7 : block2_conv1__activation__, <keras.layers.core.Activation object at 0x7efdf0682990>
8 : block2_conv2, <keras.layers.convolutional.Conv2D object at 0x7efdeda1ae50>
9 : block2_conv2__activation__, <keras.layers.core.Activation object at 0x7efdf0682410>
10 : block2_pool, <keras.layers.pooling.MaxPooling2D object at 0x7efdeda2fed0>
11 : block3_conv1, <keras.layers.convolutional.Conv2D object at 0x7efded9d49d0>
12 : block3_conv1__act

In [13]:
coreml_model.save('disdatkerasv5.mlmodel')